## W7L2 IP - Facililty Location OM 7.3

# Table of Contents<a id="Top"></a>

1. [Problem Statement](#1)<br>
2. [Data](#2) <br>
3. [Original Model Definition](#3)<br>
4. [Original Model Solution](#4)<br>
5. [Additional Constraints (Logical/Disjunctive) Model Definition](#5)<br>
6. [Additional Constraints (Logical/Disjunctive) Model Solution](#6)<br>

## 1. Problem Statement<a id=1></a>

A Facility Location problem is a blend of a transportation model with fixed costs. So we have to design the model to include rows of flow decision variables. In this example we are moving products from the 4 cities New York, Atlanta, Chicago and Los Angeles to the distribution centers in the East, South, Midwest and West. The data is givein in the Excel file `W7L1IP Logical Const Fixed Cost Disjunct.xlsx` and sheet `OM 7.3 Facility Location`.

##### [Back to Top](#Top)

## 2. Data<a id=2></a>

In [1]:
#Note you can view versions with the code !pip show matplotlib
import pandas as pd #ver 1.1.3
import pyomo.environ as pe #ver 5.7
import matplotlib.pyplot as plt #ver 3.3.2
#needed to see graphs in notebook
%matplotlib inline 

In [2]:
raw_data = pd.read_excel('W7L1IP Logical Const Fixed Cost Disjunct.xlsx', sheet_name='OM 7.3 Facility Location')
raw_data

,Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,To,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,East,South,Midwest,West,Capacity,Annual cost ($000s),NaN,NaN
2,From,New York,206,225,230,290,150,6000,NaN,NaN
3,NaN,Atlanta,225,206,221,270,150,5500,NaN,NaN
4,NaN,Chicago,230,221,208,262,150,5800,NaN,NaN
5,NaN,Los Angeles,290,270,262,215,150,6200,NaN,NaN
6,NaN,Demand,100,150,110,90,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Decisions,NaN,NaN,NaN,NaN,NaN,=SUM(C16:F16),NaN,=G7*J16,NaN


In [3]:
DV_indexes = ['E','S','M','W']
coef = pd.DataFrame(raw_data.iloc[[2,3,4,5], [2,3,4,5]])
coef.index = ['N', 'A', 'C','L']
coef.columns = DV_indexes
coef

,E,S,M,W
N,206,225,230,290
A,225,206,221,270
C,230,221,208,262
L,290,270,262,215


In [4]:
capcost = raw_data.iloc[[2,3,4,5], [6,7]]
capcost.columns = ['capacity','annualcost']
capcost.index = coef.index
capcost

,capacity,annualcost
N,150,6000
A,150,5500
C,150,5800
L,150,6200


In [5]:
demand = pd.DataFrame()
demand.loc['E','demand']=100
demand.loc['S','demand']=150
demand.loc['M','demand']=110
demand.loc['W','demand']=90
demand

,demand
E,100.0
S,150.0
M,110.0
W,90.0


##### [Back to Top](#Top)

### 3. Original Model Definition<a id=3></a>

In [6]:
model = pe.ConcreteModel()

## Define Decision Variables

Define your decision variables. __NOTE:__ for this model you have 2 sets of changing cells that you want the solver to determine - the `x` values with quantities for each path and the utilize `yU` values. Note the indexes for the `y` variables are the 4 "row" city variables.

In [7]:
model.N = pe.Var(DV_indexes, domain=pe.NonNegativeReals)
model.A = pe.Var(DV_indexes, domain=pe.NonNegativeReals)
model.C = pe.Var(DV_indexes, domain=pe.NonNegativeReals)
model.L = pe.Var(DV_indexes, domain=pe.NonNegativeReals)
model.yU = pe.Var(coef.index, domain=pe.Binary)

for DV in model.component_objects(pe.Var):
    DV.pprint()

N : Size=4, Index=N_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      E :     0 :  None :  None : False :  True : NonNegativeReals
      M :     0 :  None :  None : False :  True : NonNegativeReals
      S :     0 :  None :  None : False :  True : NonNegativeReals
      W :     0 :  None :  None : False :  True : NonNegativeReals
A : Size=4, Index=A_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      E :     0 :  None :  None : False :  True : NonNegativeReals
      M :     0 :  None :  None : False :  True : NonNegativeReals
      S :     0 :  None :  None : False :  True : NonNegativeReals
      W :     0 :  None :  None : False :  True : NonNegativeReals
C : Size=4, Index=C_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      E :     0 :  None :  None : False :  True : NonNegativeReals
      M :     0 :  None :  None : False :  True : NonNegativeReals
      S :     0 :  None :  None : False :  True : NonNegativeReals
      W :     

### Define Objective Function

Define your model objective function. Note that this will be the total costs which is calculated from the fixed costs and variable sales costs. Make sure you see how these are calculated in the Excel sheet before you try to implement here.

In [8]:
#obj funct minimize cost * (N A C L) + annual cost * yU
model.obj = pe.Objective(expr=sum(coef.loc['N',c]*model.N[c] for c in DV_indexes)
                                 + sum(coef.loc['A',c]*model.A[c] for c in DV_indexes)
                                 + sum(coef.loc['C',c]*model.C[c] for c in DV_indexes)
                                 + sum(coef.loc['L',c]*model.L[c] for c in DV_indexes)
                                 + sum(capcost.loc[c,'annualcost'] * model.yU[c] for c in coef.index),
                         sense=pe.minimize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 206*N[E] + 225*N[S] + 230*N[M] + 290*N[W] + 225*A[E] + 206*A[S] + 221*A[M] + 270*A[W] + 230*C[E] + 221*C[S] + 208*C[M] + 262*C[W] + 290*L[E] + 270*L[S] + 262*L[M] + 215*L[W] + 6000*yU[N] + 5500*yU[A] + 5800*yU[C] + 6200*yU[L]


### Define constraints

In [9]:
#Demand Constraints
model.constE = pe.Constraint(expr=model.N['E'] + model.A['E']+model.C['E'] + model.L['E'] 
                             >= demand.loc['E','demand'])
model.constS = pe.Constraint(expr=model.N['S'] + model.A['S']+model.C['S'] + model.L['S']
                             >= demand.loc['S', 'demand'])
model.constM = pe.Constraint(expr=model.N['M'] + model.A['M']+model.C['M'] + model.L['M']
                             >= demand.loc['M', 'demand'])
model.constW = pe.Constraint(expr=model.N['W'] + model.A['W']+model.C['W'] + model.L['W']
                             >= demand.loc['W', 'demand'])

# Capacity and Linking Constraints DV <= M*yU
model.constlinkN = pe.Constraint(expr=sum(model.N[c] for c in DV_indexes) 
                                 <= capcost.loc['N','capacity'] * model.yU['N'])
model.constlinkA = pe.Constraint(expr=sum(model.A[c] for c in DV_indexes) 
                                 <= capcost.loc['A','capacity']* model.yU['A'])
model.constlinkC = pe.Constraint(expr=sum(model.C[c] for c in DV_indexes) 
                                 <= capcost.loc['C','capacity'] * model.yU['C'])
model.constlinkL = pe.Constraint(expr=sum(model.L[c] for c in DV_indexes) 
                                 <= capcost.loc['L','capacity'] * model.yU['L'])

for con in model.component_objects(pe.Constraint):
    print(con,con.pprint())

constE : Size=1, Index=None, Active=True
    Key  : Lower : Body                      : Upper : Active
    None : 100.0 : N[E] + A[E] + C[E] + L[E] :  +Inf :   True
constE None
constS : Size=1, Index=None, Active=True
    Key  : Lower : Body                      : Upper : Active
    None : 150.0 : N[S] + A[S] + C[S] + L[S] :  +Inf :   True
constS None
constM : Size=1, Index=None, Active=True
    Key  : Lower : Body                      : Upper : Active
    None : 110.0 : N[M] + A[M] + C[M] + L[M] :  +Inf :   True
constM None
constW : Size=1, Index=None, Active=True
    Key  : Lower : Body                      : Upper : Active
    None :  90.0 : N[W] + A[W] + C[W] + L[W] :  +Inf :   True
constW None
constlinkN : Size=1, Index=None, Active=True
    Key  : Lower : Body                                  : Upper : Active
    None :  -Inf : N[E] + N[S] + N[M] + N[W] - 150*yU[N] :   0.0 :   True
constlinkN None
constlinkA : Size=1, Index=None, Active=True
    Key  : Lower : Body               

##### [Back to Top](#Top)

### 4. Original Model Solution<a id=4></a>

In [10]:
opt = pe.SolverFactory('glpk')
#opt.solve(model,tee=True) 
success=opt.solve(model)
print(success.solver.status,success.solver.termination_condition)

ok optimal


### Optimal Objective Value

In [11]:
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

optimal objective value minimum cost = $115770.00


### Optimal Decision Variables

In [12]:
DV_solution = pd.DataFrame()
for c in DV_indexes:
    DV_solution.loc['N',c] = model.N[c].value
    DV_solution.loc['A',c] = model.A[c].value
    DV_solution.loc['C',c] = model.C[c].value
    DV_solution.loc['L',c] = model.L[c].value
DV_solution

,E,S,M,W
N,100.0,0.0,50.0,0.0
A,0.0,150.0,0.0,0.0
C,0.0,0.0,0.0,0.0
L,0.0,0.0,60.0,90.0


We don't really care about these values - but we can do a double check that the linking variables seem to be working - here `yUC` is - which it should be because we did not use the Chicago paths.

In [13]:
for c in coef.index:
    print(f'yU{c}:',model.yU[c].value)

yUN: 1.0
yUA: 1.0
yUC: 0.0
yUL: 1.0


##### [Back to Top](#Top)

### 5. Additional Constraints (Logical/Disjunctive) Model Definition<a id=5></a>

We have already defined the needed data - so we will use these again. I am going to output them just so I can see them closer to this new model solution to remove me of their names, rows, and columns. 

In [14]:
coef

,E,S,M,W
N,206,225,230,290
A,225,206,221,270
C,230,221,208,262
L,290,270,262,215


In [15]:
capcost

,capacity,annualcost
N,150,6000
A,150,5500
C,150,5800
L,150,6200


In [16]:
demand

,demand
E,100.0
S,150.0
M,110.0
W,90.0


### Define Decision Variables

Now we already have the DV for each path quantitiy `model.x` and the Fixed Cost binary indicator `yU`. But we need to add 2 more constraints.

In [37]:
model = pe.ConcreteModel()

In [38]:
DV_indexes = ['E','S','M','W',]
model.N = pe.Var(DV_indexes, domain=pe.NonNegativeReals)
model.A = pe.Var(DV_indexes, domain=pe.NonNegativeReals)
model.C = pe.Var(DV_indexes, domain=pe.NonNegativeReals)
model.L = pe.Var(DV_indexes, domain=pe.NonNegativeReals)
model.yU = pe.Var(coef.index, domain=pe.Binary)

#BINARY Threshold NM > 60 or 0 Indicator
model.yNM60 = pe.Var(domain=pe.Binary)
#BINARY Constraint LE >= 50
model.yLE50 = pe.Var(domain=pe.Binary)
#BINARY Constraint NE <= 75
model.yNE75 = pe.Var(domain=pe.Binary)


### Define Objective Function

We are not changing the objective function with the additional constraints so this stays the same.

In [39]:
#obj funct minimize cost * (N A C L) + annual cost * yU
model.obj = pe.Objective(expr=sum(coef.loc['N',c]*model.N[c] for c in DV_indexes)
                                 + sum(coef.loc['A',c]*model.A[c] for c in DV_indexes)
                                 + sum(coef.loc['C',c]*model.C[c] for c in DV_indexes)
                                 + sum(coef.loc['L',c]*model.L[c] for c in DV_indexes)
                                 + sum(capcost.loc[c,'annualcost'] * model.yU[c] for c in coef.index),
                         sense=pe.minimize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 206*N[E] + 225*N[S] + 230*N[M] + 290*N[W] + 225*A[E] + 206*A[S] + 221*A[M] + 270*A[W] + 230*C[E] + 221*C[S] + 208*C[M] + 262*C[W] + 290*L[E] + 270*L[S] + 262*L[M] + 215*L[W] + 6000*yU[N] + 5500*yU[A] + 5800*yU[C] + 6200*yU[L]


In [40]:
#Demand Constraints
model.constE = pe.Constraint(expr=model.N['E'] + model.A['E']+model.C['E'] + model.L['E'] 
                             >= demand.loc['E','demand'])
model.constS = pe.Constraint(expr=model.N['S'] + model.A['S']+model.C['S'] + model.L['S']
                             >= demand.loc['S', 'demand'])
model.constM = pe.Constraint(expr=model.N['M'] + model.A['M']+model.C['M'] + model.L['M']
                             >= demand.loc['M', 'demand'])
model.constW = pe.Constraint(expr=model.N['W'] + model.A['W']+model.C['W'] + model.L['W']
                             >= demand.loc['W', 'demand'])

# Capacity and Linking Constraints DV <= M*yU
model.constlinkN = pe.Constraint(expr=sum(model.N[c] for c in DV_indexes) 
                                 <= capcost.loc['N','capacity'] * model.yU['N'])
model.constlinkA = pe.Constraint(expr=sum(model.A[c] for c in DV_indexes) 
                                 <= capcost.loc['A','capacity']* model.yU['A'])
model.constlinkC = pe.Constraint(expr=sum(model.C[c] for c in DV_indexes) 
                                 <= capcost.loc['C','capacity'] * model.yU['C'])
model.constlinkL = pe.Constraint(expr=sum(model.L[c] for c in DV_indexes) 
                                 <= capcost.loc['L','capacity'] * model.yU['L'])

#New constraints
big_M = 150
#Threshold NM >=60 or must be 0: NM >= 60 * yNM and NM <= 150 * yNM
model.constNM601 = pe.Constraint(expr=model.N['M'] >= 60 * model.yNM60)
model.constNM602 = pe.Constraint(expr=model.N['M'] <= 150 * model.yNM60)

#Disjunctive LE >= 50: LE−𝑀*yLE≤50 and LE+𝑀(1−𝑦LE)≥50

model.constLE501 = pe.Constraint(expr=model.L['E']-big_M*model.yLE50 <= 50)
model.constLE502 = pe.Constraint(expr=model.L['E']+big_M*(1-model.yLE50) >= 50)

#Disjunctive NE <=75: NE+𝑀*yNE≥75 and NE−𝑀(1−𝑦NE)≤75
model.constNE751 = pe.Constraint(expr=model.N['E']+big_M*model.yNE75 >= 75)
model.constNE752 = pe.Constraint(expr=model.N['E']-big_M*(1-model.yNE75) <= 75)

#Disjunctive Linking yLE + yNE >= 1
model.constLink = pe.Constraint(expr=model.yLE50 + model.yNE75 >= 1)

##### [Back to Top](#Top)

### 6. Additional Constraints (Logical/Disjunctive) Model Solution<a id=6></a>

In [41]:
opt = pe.SolverFactory('glpk')
#opt.solve(model,tee=True) 
success=opt.solve(model)
print(success.solver.status,success.solver.termination_condition)

ok optimal


### Optimal Objective Value

In [42]:
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

optimal objective value minimum cost = $116850.00


### Optimal Decision Variables

In [43]:
DV_solution = pd.DataFrame()
for c in DV_indexes:
    DV_solution.loc['N',c] = model.N[c].value
    DV_solution.loc['A',c] = model.A[c].value
    DV_solution.loc['C',c] = model.C[c].value
    DV_solution.loc['L',c] = model.L[c].value
DV_solution

,E,S,M,W
N,75.0,15.0,60.0,0.0
A,25.0,125.0,0.0,0.0
C,0.0,0.0,0.0,0.0
L,0.0,10.0,50.0,90.0


We don't really care about these values - but we can do a double check that the linking variables seem to be working.
* The `yU` variables check usage for each set of City paths. Here we see `yUC` is 0 because we did not use the Chicago paths.
* The `yLE50` checks if LE >= 50. Above we see LE final count is 0 which means it was not met which matches the y of 0
* the `yNE75` checks if NE <= 75. Above we see NE final count is 75 which means it did meet which matches the y of 1.

In [48]:
for c in coef.index:
    print(f'yU{c}:',model.yU[c].value)

yUN: 1.0
yUA: 1.0
yUC: 0.0
yUL: 1.0


In [49]:
print(f'yNM60:',model.yNM60.value)
print(f'yLE50:',model.yLE50.value)
print(f'yNE75:',model.yNE75.value)

yNM60: 1.0
yLE50: 0.0
yNE75: 1.0
